In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import time
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

# Load your Excel file (replace 'shirara_dataset.xlsx' with the correct file name and path)
df = pd.read_excel('shihara_dataset.xlsx')

# Check the first few rows of the dataframe to understand its structure
df.head()


,Date,Particulars,Payments,Receipts,Balance
0,2022-12-27,IB CEFT CHGS K A Kumaranayek,30,NaN,8449.35
1,2022-12-27,PURCHASE UNIVERSITY GRANTS C,50,NaN,8399.35
2,2022-12-28,PURCHASE KEELLS SUPER - MORA,5000,NaN,3399.35
3,2022-12-30,INTEREST,NaN,41.57,3440.92
4,2023-01-04,NOV RENT,3000,75000.00,78440.92


In [2]:
# Initialize the SentenceTransformer model (use a pre-trained model)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# You can check if the model is loaded properly
print("Model loaded successfully!")


Model loaded successfully!


In [3]:
# Convert "Particulars" column into a list of text entries
particulars_texts = df['Particulars'].astype(str).tolist()

# Generate embeddings using the model
embeddings = model.encode(particulars_texts, convert_to_numpy=True)

# Check the shape of the embeddings
print(f"Generated {embeddings.shape[0]} embeddings with dimension {embeddings.shape[1]}")

Generated 557 embeddings with dimension 384


In [4]:
import chromadb

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")  # Stores the database persistently

# Create or get the collection for banking transactions
collection_name = "banking_transactions"
collection = client.get_or_create_collection(collection_name)

print(f"Collection '{collection_name}' is ready!")


Collection 'banking_transactions' is ready!


In [ ]:
import chromadb
import uuid  # Importing uuid to generate unique IDs

# 1️⃣ Initialize ChromaDB Client
client = chromadb.PersistentClient(path="./chroma_db")  # Ensure persistent storage
collection_name = 'banking_transactions'
collection = client.get_or_create_collection(collection_name)

# 2️⃣ Debug: Check embeddings before inserting
print(f"Total embeddings generated: {len(embeddings)}")
if len(embeddings) == 0:
    raise ValueError("❌ No embeddings found! Check embedding generation step.")

print("Sample embedding:", embeddings[0])  # Print first embedding

# 3️⃣ Convert Timestamps to Strings (Fix ValueError)
df['Date'] = df['Date'].astype(str)

# 4️⃣ Insert ONLY 5 rows first to test, using UUIDs for unique IDs
try:
    # Generate unique IDs using UUID
    unique_ids = [str(uuid.uuid4()) for _ in range(5)]
    
    collection.add(
        ids=unique_ids, 
        embeddings=[emb.tolist() for emb in embeddings[:5]], 
        metadatas=df[['Date', 'Particulars', 'Payments', 'Receipts', 'Balance']].iloc[:5].to_dict(orient='records')
    )
    print("✅ Successfully inserted 5 rows into ChromaDB!")
except Exception as e:
    print(f"❌ Error inserting data: {e}")

# 5️⃣ Check if ChromaDB Stored Data
print("Total documents in collection:", collection.count())
print("Sample document from ChromaDB:", collection.peek(1))


In [1]:



# Generate embeddings for the 'Particulars' column
embeddings = model.encode(df['Particulars'].tolist(), show_progress_bar=True)

# Step 3: Reduce the dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # You can adjust the number of components (e.g., 50)
reduced_embeddings = pca.fit_transform(embeddings)

# Step 4: Apply MiniBatchKMeans clustering on the reduced embeddings
n_clusters = 5  # You can adjust the number of clusters

# Track the start time for clustering
start_time = time.time()

# Initialize MiniBatchKMeans
mini_batch_kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=100)
labels = mini_batch_kmeans.fit_predict(reduced_embeddings)

# Track the end time for clustering
end_time = time.time()
print(f"Clustering took {end_time - start_time} seconds")

# Step 5: Add the cluster labels to the DataFrame
df['Cluster'] = labels

# Step 6: Print the results for each cluster
for cluster_num in range(n_clusters):
    print(f"\n--- Cluster {cluster_num} ---")
    cluster_transactions = df[df['Cluster'] == cluster_num]
    print(cluster_transactions[['Date', 'Particulars', 'Payments', 'Receipts', 'Balance']])
    print("\n")


NameError: name 'df' is not defined